In [1]:
import numpy as np 
import pandas as pd
import sklearn
from sklearn.preprocessing import minmax_scale
import sys
import os
import random 

# Load the submitted codes
import submission_c0 as c0 # Cristin's algorithm 2020-01-23 12:24:41.577445+00:00 
import submission_c1 as c1 # Cristin's algorithm 2020-01-22 04:42:14.732860+00:00
import submission_j0 as j0 # John's algorithm 2020-02-05 15:07:31.838107+00:00
import submission_j1 as j1 # John's algorithm 2020-01-14 07:31:21.625687+00:00


In [2]:

def slide_window(target,algs,window=5): # sample 5 subsets from the dataset and test the algorithms
    source_dir="./public_dat/"+str(target)+"/"
    try:
        os.mkdir('./sliding_window/')
    except :
        pass
    target_dir='./sliding_window/'
    sub_entries=os.listdir(source_dir)
    sub_entries.sort()
    fragments=[]
    if('.DS_Store' in sub_entries):
        sub_entries.remove('.DS_Store')
    scores=np.zeros(len(algs))
    count=0
    while(len(sub_entries)>1):
        count+=1
        sample=random.sample(sub_entries,window)
        for i,file in enumerate(sample):
            sub_entries.remove(file)
            custom_data=pd.read_csv(source_dir+file,encoding='utf-8')

            x=custom_data['x'].values
            x = minmax_scale(x)
            custom_data['x']=x.reshape(-1)
            y=custom_data['y'].values
            y = minmax_scale(y)
            custom_data['y']=y.reshape(-1)
            t=custom_data['t'].values
            min_elem=t[0]
            t=[x-min_elem for x in t]
            custom_data['t']=t
            custom_data=custom_data.rename(columns={0:"t",1:"x",2:"y"}) 
            custom_data.to_csv(target_dir+str(i)+".csv",index=False)
            
        for i,elem in enumerate(algs):
            scores[i]+=elem(target_dir)
            
  
    for w in range(len(scores)):
        scores[w]=scores[w]/count
    return scores

## Cristin

### Scoring Cristin's Algorithms

#### Seal Diving Dataset

In [4]:
algs=[c0.predictor,c1.predictor]
scores_cristin_seal=np.zeros(len(algs))
for i in range(100):
    scores_cristin_seal+=slide_window("10",algs)
scores_cristin_seal/=np.ones(len(algs))*100
scores_cristin_seal

array([0.9175, 1.    ])

#### Drone Dataset

In [3]:
algs=[c0.predictor,c1.predictor]
scores_cristin_drone=np.zeros(len(algs))
for i in range(100):
    scores_cristin_drone+=slide_window("11",algs)
scores_cristin_drone/=np.ones(len(algs))*100
scores_cristin_drone=np.ones(len(algs))-scores_cristin_drone
scores_cristin_drone

array([1., 0.])

## John

### Scoring John's Algorithms

#### Seal diving Dataset

In [5]:
algs=[j0.predictor,j1.predictor]
scores_john_seal=np.zeros(len(algs))
for i in range(100):
    scores_john_seal+=slide_window("10",algs)
scores_john_seal/=np.ones(len(algs))*100
scores_john_seal    

array([0.17625, 0.70125])

#### Drone Dataset

In [6]:
algs=[j0.predictor,j1.predictor]
scores_john_drone=np.zeros(len(algs))
for i in range(100):
    scores_john_drone+=slide_window("11",algs)
scores_john_drone/=np.ones(len(algs))*100
scores_john_drone=np.ones(len(algs))-scores_john_drone
scores_john_drone

array([0.97, 0.34])

## Sum Up

In [7]:
john_drone=np.append(scores_john_drone,np.average(scores_john_drone[:2]))
christin_drone=np.append(scores_cristin_drone,np.average(scores_cristin_drone[:2]))

john_seal=np.append(scores_john_seal,np.average(scores_john_seal[:2]))
christin_seal=np.append(scores_cristin_seal,np.average(scores_cristin_seal[:2]))


d = {'submission':['0','1','average'],'john drone ': john_drone, 'Cristin drone ': christin_drone,'john seal ': john_seal,'Cristin seal ': christin_seal}
df = pd.DataFrame(data=d)
print("Average score \nJohn: "+str((john_seal[-1]+john_drone[-1])/2)+"\nCristin: "+str((christin_seal[-1]+christin_drone[-1])/2)+"")
df

Average score 
John: 0.546875
Cristin: 0.729375


,submission,john drone,Cristin drone,john seal,Cristin seal
0,0,0.970,1.0,0.17625,0.91750
1,1,0.340,0.0,0.70125,1.00000
2,average,0.655,0.5,0.43875,0.95875
